In [ ]:
import src.peaks.api as ppi
from src.peaks.refinder import RePeakFinder
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import src.vae.api as vpi
import random
import src.generator.api as gpi

# dates = vpi.API().unique_dates()

In [ ]:
synthetic_keys = vpi.API().unique_dates()

In [ ]:
random.shuffle(synthetic_keys)
synthetics = gpi.API().synthetics(keys=synthetic_keys[0:1500])

In [ ]:
dates = ppi.API().unique_dates()
meas = ppi.API().measurement(dates)

In [ ]:
stds = meas.loc[meas["energy"] >= 500].groupby("datetime").std(numeric_only=True)["count"]

# Step 2: Compute desired percentiles
percentiles = stds.quantile([0.0, 0.25, 0.5, 0.75, 0.99])

# Step 3: Find datetimes closest to each percentile
closest_datetimes = {}
for q, target in percentiles.items():
    closest = (stds - target).abs().idxmin()
    closest_datetimes[q] = closest

# Optional: Show result
for q, dt in closest_datetimes.items():
    print(f"{int(q * 100)}th percentile → {dt} (std ≈ {stds[dt]:.4f})")

In [ ]:
# Step 1: Group by datetime and compute std
stds = meas.loc[(meas["energy"] < 500) & (meas["energy"] > 20)].groupby("datetime").std(numeric_only=True)["count"]

# Step 2: Compute desired percentiles
percentiles = stds.quantile([0.0, 0.25, 0.5, 0.75, 0.99])

# Step 3: Find datetimes closest to each percentile
closest_datetimes = {}
for q, target in percentiles.items():
    closest = (stds - target).abs().idxmin()
    closest_datetimes[q] = closest

# Optional: Show result
for q, dt in closest_datetimes.items():
    print(f"{int(q * 100)}th percentile → {dt} (std ≈ {stds[dt]:.4f})")

In [ ]:
# Step 1: Group by datetime and compute std
stds = synthetics.loc[synthetics["energy"] >= 500].groupby("datetime").std(numeric_only=True)["count"]

# Step 2: Compute desired percentiles
percentiles = stds.quantile([0.0, 0.25, 0.5, 0.75, 0.99])

# Step 3: Find datetimes closest to each percentile
closest_datetimes = {}
for q, target in percentiles.items():
    closest = (stds - target).abs().idxmin()
    closest_datetimes[q] = closest

# Optional: Show result
for q, dt in closest_datetimes.items():
    print(f"{int(q * 100)}th percentile → {dt} (std ≈ {stds[dt]:.4f})")

In [ ]:
# Step 1: Group by datetime and compute std
stds = \
synthetics.loc[(synthetics["energy"] < 500) & (synthetics["energy"] > 20)].groupby("datetime").std(numeric_only=True)[
    "count"]

# Step 2: Compute desired percentiles
percentiles = stds.quantile([0.0, 0.25, 0.5, 0.75, 0.99])

# Step 3: Find datetimes closest to each percentile
closest_datetimes = {}
for q, target in percentiles.items():
    closest = (stds - target).abs().idxmin()
    closest_datetimes[q] = closest

# Optional: Show result
for q, dt in closest_datetimes.items():
    print(f"{int(q * 100)}th percentile → {dt} (std ≈ {stds[dt]:.4f})")

In [ ]:
# dates = ppi.API().unique_dates()
# meas = ppi.API().measurement([dates[500]])


to_analze = synthetics.loc[synthetics["datetime"] == "v53_synthetic_1312_2019-11-27 08:58:52"].reset_index(drop=True)



In [ ]:
import plotly.graph_objects as go

res = RePeakFinder(
    selected_date=None,
    data=to_analze[["datetime", "energy", "count"]],
    meta=None,
    schema="re_processed_synthetics",
    matching_ratio=0,
    interpolate_energy=False,
    measurement_peaks_prefix=""
).process_spectrum(return_detailed_view=False)
res["counts_cleaned"] = res["count"] - res["background"]
peaks = res.loc[res["peak"] == True]

fig = go.Figure()

# Raw counts
fig.add_trace(go.Scatter(
    x=res["energy"],
    y=res["count"],
    mode='lines',
    name='Raw Counts'
))

# Peak positions
fig.add_trace(go.Scatter(
    x=peaks["energy"],
    y=peaks["count"],
    mode='markers',
    text=peaks["identified_isotope"],
    marker=dict(color='black', size=8, symbol='x')
))

from scipy.signal import savgol_filter

y_smooth = savgol_filter(res["count"].to_numpy(), window_length=8, polyorder=1)
fig.add_trace(go.Scatter(
    x=res["energy"],
    y=y_smooth,
    mode='lines',
    name='Raw Counts'
))

fig.update_layout(
    title="Peak Detection Overlay",
    xaxis_title="Energy",
    yaxis_title="Counts",
    template="plotly_white"
)

fig.show()

In [ ]:
# MODELL FÜR > 500keV
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# real_greater500
x_real_greater500 = np.array([4, 1, 1.4, 0.2, 0.02])
y_real_greater500 = np.array([0.3507, 2.5214, 5.8117, 18.8154, 2417.0879])

# real_less500
x_real_less500 = np.array([0.2, 0.7, 0.8, 0.6, 0.1])
y_real_less500 = np.array([1.21424, 10.7187, 15.6976, 48.2542, 11026.2624])

# synthetic_greater500
x_synthetic_greater500 = np.array([0.2, 0.4, 0.7, 0.2, 0.1])
y_synthetic_greater500 = np.array([0.1161, 6.0469, 14.7788, 36.3063, 5596.2659])

# synthetic_less500
x_synthetic_less500 = np.array([0.01, 0.086, 0.1, 0.4, 1])
y_synthetic_less500 = np.array([0.3163, 19.2846, 53.5015, 146.7614, 31982.5049])

def create_model(x, y):
    def exp_model(x, a, b):
        return a * np.exp(-b * x)

    # Fit
    popt, _ = curve_fit(exp_model, x, y, p0=(2000, 2), maxfev=20000)
    a, b = popt

    # Vorhersagekurve
    x_fit = np.linspace(0, 1, 200)
    y_fit = exp_model(x_fit, a, b)

    # Plot
    plt.scatter(x, y, label="Original Data", color="red")
    plt.plot(x_fit, y_fit, label=f"Fit: y = {a:.1f} * exp(-{b:.2f} * x)", color="blue")
    plt.xlabel("std")
    plt.ylabel("Threshold")
    plt.legend()
    plt.grid(True)
    plt.ylim(-100,13000)
    plt.title("Exponentielles Modell für Threshold vs. std")
    plt.show()


create_model(x_synthetic_greater500, y_synthetic_greater500)

In [ ]:
std = 7
x = np.array([0.2, 0.4, 0.7, 0.2, 0.1]) # _greater500
y = np.array([0.1161, 6.0469, 14.7788, 36.3063, 5596.2659]) # _greater500

if std > y[-1]:
    estimated_prominance = x[-1]
if std < y[0]:
    estimated_prominance = x[0]

else:
    min_prominence = x[std >= y][-1]
    max_prominence = x[std <= y][0]
    estimated_prominance = np.mean([min_prominence, max_prominence])

print(estimated_prominance)


